In [1]:
import argparse
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms.transforms import ToTensor

In [18]:
class ImageDataset(Dataset):
    def __init__(self, images, labels):
        super().__init__()
        self.images, self.labels = images, labels

    def __len__(self):
        return self.images.shape[0]

    def __getitm__(self, idx):
        image, label = self.images[idx], self.labels[idx]
        # image = torch.FloatTensor(image)/255.
        # label = torch.LongTensor(label)
        # return x.view(-1, 784), y
        return image, label


def get_dataloaders(batch_size, test_ratio=0.8):
    import mnist
    from sklearn.model_selection import train_test_split

    images, labels = mnist.train_images(), mnist.train_labels()
    train_images, valid_images, train_labels, valid_labels = train_test_split(
            images, labels, test_size=test_ratio, random_state=111)
    test_images, test_labels = mnist.test_images(), mnist.test_labels()

    train_dataset = ImageDataset(train_images, train_labels)
    valid_dataset = ImageDataset(valid_images, valid_labels)
    test_dataset  = ImageDataset(test_images,  test_labels)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

    return train_loader, valid_loader, test_loader

In [19]:
## Hyper parameters
n_epochs = 10
batch_size = 256

train_loader, valid_loader, test_loader = get_dataloaders(batch_size)

In [20]:
x_train, y_train = train_loader.dataset.images, train_loader.dataset.labels
x_valid, y_valid = valid_loader.dataset.images, valid_loader.dataset.labels

print("Train images:", type(x_train), x_train.shape, x_train.dtype)
print("Train labels:", type(y_train), y_train.shape, y_train.dtype)
print("Valid images:", type(x_valid), x_valid.shape, x_valid.dtype)
print("Valid labels:", type(y_valid), y_valid.shape, y_valid.dtype)

Train images: <class 'numpy.ndarray'> (12000, 28, 28) uint8
Train labels: <class 'numpy.ndarray'> (12000,) uint8
Valid images: <class 'numpy.ndarray'> (48000, 28, 28) uint8
Valid labels: <class 'numpy.ndarray'> (48000,) uint8


In [26]:
import mnist
from sklearn.model_selection import train_test_split

images, labels = mnist.train_images(), mnist.train_labels()
train_images, valid_images, train_labels, valid_labels = train_test_split(
        images, labels, test_size=0.8, random_state=111)

print(images.shape, labels.shape)

(60000, 28, 28) (60000,)


In [30]:
train_dataset = ImageDataset(train_images, train_labels)

train_dataset.images[0].shape

(28, 28)

In [32]:
from torchvision import datasets, transforms

train_set = datasets.MNIST('data', train=True, download=True,
        transform=transforms.ToTensor())

train_loader = DataLoader(train_set, batch_size=256, shuffle=True)

x, y= next(iter(train_loader))
x.shape, y.shape

(torch.Size([256, 1, 28, 28]), torch.Size([256]))

In [91]:
import mnist

class ImageDataset(Dataset):
    def __init__(self, images, labels):
        super().__init__()
        self.images, self.labels = images, labels

    def __len__(self):
        return self.images.shape[0]

    def __getitem__(self, idx):
        image, label = self.images[idx], self.labels[idx]
        return image, label


def get_dataloader(images, labels, batch_size, shuffle=False):

    ## Preprocessing
    images = torch.from_numpy(images).view(-1, 784).float()/255.
    labels = torch.from_numpy(labels).long()
    
    dataloader = DataLoader(
        dataset=ImageDataset(images, labels),
        batch_size=batch_size,
        shuffle=shuffle)
    
    return dataloader

images, labels = mnist.train_images(), mnist.train_labels()
train_images, valid_images, train_labels, valid_labels = train_test_split(
            images, labels, test_size=0.8, random_state=111)
test_images, test_labels = mnist.test_images(), mnist.test_labels()

batch_size = 256
train_loader = get_dataloader(train_images, train_labels, batch_size, shuffle=True)
valid_loader = get_dataloader(valid_images, valid_labels, batch_size)
test_loader  = get_dataloader(test_images, test_labels, batch_size)

x, y = next(iter(test_loader))
print(x.shape, x.dtype, x.device)
print(y.shape, y.dtype, y.device)



torch.Size([256, 784]) torch.float32 cpu
torch.Size([256]) torch.int64 cpu


In [86]:
input_size, output_size = 784, 10

model = nn.Sequential(
    nn.Linear(input_size, 200),
    nn.ReLU(),
    nn.Linear(200, 200),
    nn.ReLU(),
    nn.Linear(200, output_size),
    nn.LogSoftmax(dim=-1),
)

loss_fn = nn.NLLLoss()
optim = torch.optim.Adam(model.parameters())

y_hat = model(x)
y.shape, y_hat.shape

(torch.Size([256]), torch.Size([256, 10]))

In [90]:
loss_fn(y_hat, y.squeeze()).item()

2.3109660148620605